In [ ]:
# import libraries
try:
  %tensorflow_version
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import losses

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data
column_names = ["label", "text"]

train_dataset = pd.read_csv(train_file_path, sep="\t", names=column_names)
test_dataset = pd.read_csv(test_file_path, sep="\t", names=column_names)
train_dataset.head()

In [ ]:
# Statistics about text length
text_lengths = train_dataset["text"].apply(len)
min_length = text_lengths.min()
max_length = text_lengths.max()
average_length = text_lengths.mean()
print("Minimum Length:", min_length)
print("Maximum Length:", max_length)
print("Average Length:", average_length)

In [ ]:
# Map labels to integers
label_mapping = {'ham': 0, 'spam': 1}
reverse_label_mapping = {0: 'ham', 1: 'spam'}

train_dataset['label'] = train_dataset['label'].map(label_mapping)
test_dataset['label'] = test_dataset['label'].map(label_mapping)

train_dataset.head()

In [ ]:
# Split labels from dataset
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

print(len(train_features), len(test_features))

In [ ]:
train_features.head()

In [ ]:
train_labels.head()

In [ ]:
# Create vectorization layer
max_features = 10000
sequence_length = 150

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)
vectorize_layer.adapt(train_features['text'].values)

In [ ]:
# Vectorize dataset
train_vectorized = vectorize_layer(train_features['text'].values)
test_vectorized = vectorize_layer(test_features['text'].values)

In [ ]:
# Define model
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

model.summary()

In [ ]:
# Train model
%%time
history = model.fit(
    train_vectorized,
    train_labels,
    epochs=100,
    validation_split=0.2
    )

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
def plot_accuracy(history):
  plt.plot(history.history['binary_accuracy'], label='accuracy')
  plt.plot(history.history['val_binary_accuracy'], label='val_accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.grid(True)

  plt.show()

plot_accuracy(history)

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(test_vectorized, test_labels)

In [ ]:
# Create model for inference on text data directly
export_model = Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = export_model.predict([pred_text])[0][0]
  class_name = reverse_label_mapping[round(prediction)]

  return (prediction, class_name)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
